In [ ]:
pip install -U deepeval

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 696.0/696.0 kB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.5/357.5 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 95.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: opentelemetry-proto
    Found existing installation: opentelemetry-proto 1.37.0
    Uninstalling open

In [ ]:
from datasets import load_dataset
import deepeval

In [ ]:
# Step 1: Load the Hugging Face dataset
squad_dataset = load_dataset("squad")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

plain_text/validation-00000-of-00001.par(…):   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [ ]:
# Step 2: Prepare data for relevancy testing (e.g., question-context pairs)
data_samples = [{
    "input": sample["context"],
    "question": sample["question"],
    "expected_output": sample["answers"]["text"][0]
} for sample in squad_dataset["validation"].select(range(100))]  # Taking a subset for simplicity using .select()

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
import subprocess
import time

# Start Ollama in the background
subprocess.Popen(["ollama", "serve"])

# Give it a few seconds to start up
time.sleep(5)

In [ ]:
!ollama pull llama3:latest

In [ ]:
from datasets import load_dataset
from deepeval import evaluate
from deepeval.test_case import LLMTestCase, LLMTestCaseParams
from deepeval.metrics import GEval
from deepeval.models.base_model import DeepEvalBaseLLM
import requests

# Ollama Model for DeepEval
class OllamaModel(DeepEvalBaseLLM):
    def __init__(self, model_name="llama3:latest"):
        self.model_name = model_name
        self.ollama_url = "http://localhost:11434/api/chat"

    def load_model(self):
        return self.model_name

    def generate(self, prompt: str) -> str:
        payload = {
            "model": self.model_name,
            "messages": [
                {"role": "user", "content": prompt}
            ],
            "stream": False,
            "options": {
                "temperature": 0.7
            }
        }

        try:
            response = requests.post(self.ollama_url, json=payload, timeout=60)
            if response.status_code == 200:
                result = response.json()["message"]["content"]
                return result.strip()
            else:
                raise Exception(f"Ollama API error: {response.status_code}")
        except Exception as e:
            print(f"Error generating response: {e}")
            return "Error generating response"

    async def a_generate(self, prompt: str) -> str:
        return self.generate(prompt)

    def get_model_name(self):
        return self.model_name


# Initialize Llama 3 model
llama_model = OllamaModel("llama3:latest")

# Load Hugging Face dataset
print("Loading dataset...")
dataset = load_dataset("squad", split="validation[:10]")  # Start small

# Generate predictions
print("Generating predictions with Llama 3...")
test_cases = []

for i, item in enumerate(dataset):
    question = item['question']
    context = item['context']
    expected_answer = item['answers']['text'][0] if item['answers']['text'] else ""

    # Create prompt for Llama 3
    prompt = f"""Based on the following context, answer the question concisely.

Context: {context}

Question: {question}

Answer:"""

    # Generate answer
    actual_output = llama_model.generate(prompt)

    # Create test case
    test_case = LLMTestCase(
        input=question,
        actual_output=actual_output,
        expected_output=expected_answer,
        context=[context]
    )
    test_cases.append(test_case)

    print(f"Processed {i + 1}/{len(dataset)} examples")

# FIXED: Use LLMTestCaseParams enum for evaluation_params
print("\nSetting up evaluation metrics...")

correctness_metric = GEval(
    name="Correctness",
    criteria="Determine whether the actual output is factually correct compared to the expected output.",
    evaluation_params=[
        LLMTestCaseParams.INPUT,
        LLMTestCaseParams.ACTUAL_OUTPUT,
        LLMTestCaseParams.EXPECTED_OUTPUT
    ],
    model=llama_model,
    threshold=0.5
)

# Run evaluation
print("\nRunning evaluation...")
results = evaluate(
    test_cases=test_cases,
    metrics=[correctness_metric]
    # run_async=False
)

print("\nEvaluation complete!")

# Print results
print("\n=== Results ===")
for i, test_case in enumerate(test_cases):
    print(f"\nExample {i+1}:")
    print(f"Question: {test_case.input[:100]}...")
    print(f"Expected: {test_case.expected_output}")
    print(f"Actual: {test_case.actual_output}")
    if hasattr(test_case, 'success'):
        print(f"Success: {test_case.success}")

Loading dataset...
Generating predictions with Llama 3...
Processed 1/10 examples
Processed 2/10 examples
Processed 3/10 examples
Processed 4/10 examples
Processed 5/10 examples
Processed 6/10 examples
Processed 7/10 examples
Processed 8/10 examples
Processed 9/10 examples
Processed 10/10 examples

Setting up evaluation metrics...

Running evaluation...


✨ You're running DeepEval's latest Correctness [GEval] Metric! (using llama3:latest, strict=False, 
async_mode=True)...

Output()

INFO:deepeval.evaluate.execute:in _a_execute_llm_test_cases
INFO:deepeval.evaluate.execute:in _a_execute_llm_test_cases
INFO:deepeval.evaluate.execute:in _a_execute_llm_test_cases
INFO:deepeval.evaluate.execute:in _a_execute_llm_test_cases
INFO:deepeval.evaluate.execute:in _a_execute_llm_test_cases
INFO:deepeval.evaluate.execute:in _a_execute_llm_test_cases
INFO:deepeval.evaluate.execute:in _a_execute_llm_test_cases
INFO:deepeval.evaluate.execute:in _a_execute_llm_test_cases
INFO:deepeval.evaluate.execute:in _a_execute_llm_test_cases
INFO:deepeval.evaluate.execute:in _a_execute_llm_test_cases




Metrics Summary

  - ✅ Correctness [GEval] (score: 0.9, threshold: 0.5, strict: False, evaluation model: llama3:latest, reason: The input correctly generates the actual output, which is factually correct. However, there are minor variations in formatting or wording between the actual output and expected output that prevent a perfect match., error: None)

For test case:

  - input: Which NFL team represented the AFC at Super Bowl 50?
  - actual output: The Denver Broncos.
  - expected output: Denver Broncos
  - context: ['Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the leagu

⚠ WARNING: No hyperparameters logged.
» ]8;id=461059;https://deepeval.com/docs/evaluation-prompts\Log hyperparameters]8;;\ to attribute prompts and models to your test runs.

================================================================================

✓ Evaluation completed 🎉! (time taken: 47.94s | token cost: None USD)
» Test Results (10 total tests):
   » Pass Rate: 40.0% | Passed: 4 | Failed: 6

 ================================================================================ 

» Want to share evals with your team, or a place for your test cases to live? ❤️ 🏡
  » Run 'deepeval view' to analyze and save testing results on Confident AI.


Evaluation complete!

=== Results ===

Example 1:
Question: Which NFL team represented the AFC at Super Bowl 50?...
Expected: Denver Broncos
Actual: The Denver Broncos.

Example 2:
Question: Which NFL team represented the NFC at Super Bowl 50?...
Expected: Carolina Panthers
Actual: The Carolina Panthers.

Example 3:
Question: Where did Super Bowl 50 take place?...
Expected: Santa Clara, California
Actual: Levi's Stadium in Santa Clara, California.

Example 4:
Question: Which NFL team won Super Bowl 50?...
Expected: Denver Broncos
Actual: The Denver Broncos.

Example 5:
Question: What color was used to emphasize the 50th anniversary of the Super Bowl?...
Expected: gold
Actual: Gold.

Example 6:
Question: What was the theme of Super Bowl 50?...
Expected: "golden anniversary"
Actual: The theme of Super Bowl 50 was the "golden anniversary" with various gold-themed initiatives.

Example 7:
Question: What day was the game played on?...
Expected: February 7, 2016
Actual: February 7, 2016.

E